In [3]:
import numpy as np
import pandas as pd
import torch
import gpytorch
import math
import matplotlib.pyplot as plt
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import UnwhitenedVariationalStrategy
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from gpytorch.utils.quadrature import GaussHermiteQuadrature1D


class GPClassificationModel(ApproximateGP):
    
    
    def __init__(self,inducing_points):
        
        
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = UnwhitenedVariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super(GPClassificationModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
       

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


In [4]:
df = pd.read_csv("./data/banana.csv")
df["Class"] = (df["Class"]+1)/2

X = torch.tensor(df.iloc[:,:2].values,dtype=torch.float32)
y = torch.tensor(df.iloc[:,2].values,dtype=torch.float32)

In [15]:
torch.manual_seed(321)
np.random.seed(321)

folds = KFold(10)
splits = folds.split(X)

i = 0

lls = []
f1s = []

for train, test in splits:
    
    Xtrain = X[train,:]
    
    Xtrain_mean = np.mean(Xtrain.numpy(),0)
    Xtrain_std = np.std(Xtrain.numpy(),0)
    Xtrain = (Xtrain - Xtrain_mean) / Xtrain_std
    
    
    Xtest = X[test,:]
    Xtest = (Xtest - Xtrain_mean) / Xtrain_std
    
    ytrain = y[train]
    ytest = y[test]
    
    Xm = np.mean(X.numpy(),0)
    Xs = np.std(X.numpy(),0)

    model = GPClassificationModel((X[:10,:]-Xm)/Xs)


    likelihood = gpytorch.likelihoods.BernoulliLikelihood()
    model.train()
    likelihood.train()
        
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=len(Xtrain))
    
    for j in range(350):
        optimizer.zero_grad()

        samp = np.random.choice(len(Xtrain),100,replace=True)
        Xsamp = Xtrain[samp,:]
        ysamp = ytrain[samp]
        
        output = model(Xsamp)
        loss = -mll(output, ysamp)
        loss.backward()
    
        optimizer.step()
    
    
    norm = torch.distributions.normal.Normal(0,1)
    ll = torch.mean(GaussHermiteQuadrature1D()(lambda x: ytest*torch.log(norm.cdf(x)+1e-6)+(1-ytest)*torch.log((1-norm.cdf(x)+1e-6)),model(Xtest))).detach().numpy()
    lls.append(ll)
    
    f1 = f1_score(ytest.detach().numpy(),np.round(GaussHermiteQuadrature1D()(lambda x: norm.cdf(x),model(Xtest)).detach().numpy()))
    f1s.append(f1)


    i += 1
    print(i)

1
2
3
4
5
6
7
8
9
10


In [16]:
np.mean(lls)

-0.2717239

In [17]:
np.std(lls)

0.01859266

In [18]:
np.mean(f1s)

0.8717704809961146

In [19]:
np.std(f1s)

0.014163429876978531